<a href="https://colab.research.google.com/github/lmoss/onesharp/blob/main/universal/universal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Universal programs

We discuss <i>universal programs</i>:
these are programs that take a program as an inputs and (act as if they) run
the input program.
 


In [ ]:
#@title
!python -m pip install -U setuptools
!python -m pip install -U git+https://github.com/lmoss/onesharp.git@main
from onesharp.interpreter.interpreter import *


It can be argued that the $\onesharp$-computable functions include basically every function that anyone could imagine computing in any way.
For example, any function which could be computed on any actual computer could be programmed in $\onehash$.   Later, we will make the claim that


```{admonition} Idea
:class: tip
Any function $f:\words\to\words$ for which anyone could write clear and explicit instructions is  $\onesharp$-computable.
``` 

This claim might be called the *Church-Turing thesis* for $\onesharp$.
 Now one of the tasks that someone could 
write clear and explicit instructions for is the task of executing a given $\onesharp$ program.  So our claim above implies that 


```{admonition} Slogan
:class: tip

There is a $\onesharp$ program that
executes $\onesharp$ programs. 
```


This *follows* from the Church-Turing thesis.  But we are going to prove it in detail in this chapter.  We do this for several reasons.  First, the  Church-Turing thesis is just that:
*thesis* -- that is, a *claim*  -- that really cannot be completely proved.  We discuss this later, in Section 5.6.  So since we want to use the existence of programs that execute programs,
it would be better to firmly establish this by exhibiting such a program.   Second, there is something to be learned from writing a long program in a low-level language like $\onehash$.  
These would be missing if we used the *hand-waving approach* of appealing to the  Church-Turing thesis.   The reason that people frequently prefer to use the hand-waving approach in computability theory is that to be explicit often involves a lot of details that are "as expected",
or tedious, or which even obscure what is "really going on" in a result.   This brings us to our third reason for giving the details on programs that run programs: we're doing this in full detail so that afterwards we won't do this again.   After doing the hard work of writing a long program that does something that might be "obvious":, you will have earned the right to use a hand-waving approach.


```{admonition} Definition
:class: attention

A word $\uprog$ is *universal* if

$$
\semantics{\uprog} (x) \simeq [\![x]\!](\ )
$$

for all words $x$.
```

This means that if running $x$ on all empty registers eventually gives some output in R1 (and all other registers are empty), then that same output would be the result of running $\uprog$ with $p$ in R1 and all other registers empty. In the other direction, *if* running $x$ with all empty registers gives some output, say $y$, in R1 (and all registers empty at the end), then we would get the same output $y$ in R1 when we run $\uprog$ with $x$ in R1 and all other registers empty.

Let's take a look at such a program and try it out.

In [ ]:
u = universal


```{admonition} Examples

$$\semantics{\uprog}(\onett\hash) \simeq \one$ because $[\![\onett\hash]\!](\ )\simeq \onett$.

$$\semantics{\uprog}(\onett)\!\uparrow$  because $\one$ is not a program.
```

```{admonition} More examples
Try to figure out what the next two should be and then confirm your guesses by running them.

$\semantics{\uprog}(\one\hash\one\hash\hash)$


$\semantics{\uprog}(\one\hash \one\hash\hash \one\hash \one\hash \hash  \one\hash \hash+ \semantics{\uprog})$
 
```



### Let's write a universal program


The goal of this section is for you to write a universal program
yourself, either alone or with others.   

We hasten to make two remarks: first of all, it would be more
instructive for you to forget about the rest of this lesson 
entirely and to just write your own universal program.   But 
 most readers would probably appreciate the hints and will find
enough of a challenge in the construction of a long working program.

Second, the sections to come do not guide you to write a program that
is *exactly* like $\uprog$ above.   The program $\uprog$ 
above was designed to be as short as possible.
In fact, we leave you with a challenge: write a universal
program with the smallest number of instructions.  The one above has 
300.

### Simplification

We are going to simplify the requirements in order to make
it easier to write a universal program.  So we'll outline how
to write a program that is <i>close</i> to a universal program,
but not quite a universal program.

We'll write a program $\uprog$ that acts like a universal
program, but *only for programs which use only R1, R2, and R3*.
In more detail:
 
Let $p$ be any
program of ```1#``` which only uses R1, R2, and R3.
Let $x$ be any word on our alphabet ```{1,#}```,
including possibly the empty word. Then the following are equivalent: 

(A')
If $p$ is started with all empty registers, then eventually we halt
with $x$ in R1. 

(B') If $\uprog$ is started with $p$ in R1 and with the empty word 
in all other
registers, then eventually we halt with $x$ in R1. 

Once again, the work of this section sketches a construction of 
such a program $\uprog$.   You will need to do much of the work yourself
in actually getting the program from the sketch.  And even when we
have $\uprog$, it will not be what we really want, since
the resulting program $\uprog$ will only behave right on programs which use the first
three registers.  
Later on, we'll come
back and drop this simplification to get a program which is truly 
universal.




We are going to use <i>pseudo-registers</i> as follows:

 <CENTER>
<TABLE BORDER  Cellpadding="3">
 <TR><TD BGCOLOR="#B0E0E6">R1: the input program <i>p</i>  </TD>   </tr>
 <TR><TD BGCOLOR="#B0E0E6">R2: an instruction number 1<sup>n</sup>
        </TD>   
</tr>
 <TR><TD BGCOLOR="#B0E0E6">R3: the <i>n</i>th instruction of <i>p</i>    
</TD>   </tr>
 <TR><TD BGCOLOR="#B0E0E6">R4: the contents of the real R1   </TD>   </tr>
 <TR><TD BGCOLOR="#B0E0E6">R5: the contents of the real R2  </TD>   </tr>
 <TR><TD BGCOLOR="#B0E0E6">R6: the contents of the real R3  </TD>   </tr>
</table>
</center>
<br><p>
That is, we are going to <strong>simulate</strong> the 
computation of a register machine on a program <i>p</i> by
using six registers.   Our universal program <i>u</i> 
mimics what a register machine would do.
The only difference is that one step of a real register machine
would be done by *many* steps of our universal program.


The registers above hold what is sometimes called a <strong>snapshot
of a register machine run</strong>. 
This means that they hold all of the relevant information about a register
machine computation at one particular point in time: they tell what program is
running, which instruction would be run next and what number instruction it is,
and also the contents of all of the registers that the program is using.
  
<br><p>
Our universal program is composed of a few sub-programs.
We shall illustrate the ideas behind several sub-programs by making a
large chart.   This chart shows just one example
of how a universal program could work, when we take
<i>p</i>
to be the program  

$$ 
\begin{array}{lcl} p &  = & \mathtt{1\#\#}  +  \mathtt{write}\\
&  = & \mathtt{1\#1\#\#\#\#\#111111111\#\#\#11111\#\#\#11\#11\#\#11\#\#111111\#\#\#\#11\#11\#\#111111111\#\#\#\#11\#\#\#\#\#111111\#\#\#111\#\#\#1\#\#1111\#\#\#\#1\#111111\#\#\#\#} 
\end{array}
$$

 
<br>
<p>
Let's parse this program and number the 
instructions:
<center>
<TABLE BORDER  Cellpadding="3">
<tr>
<td> 1 </td>
<td> 2 </td>
<td> 3 </td>
<td> 4 </td>
<td> 5 </td>
<td> 6 </td>
<td> 7 </td>
<td> 8 </td>
<td> 9 </td>
</tr>
<tr BGCOLOR="#FFFFCC">
<td> 1## </td>
<td> 1##### </td>
<td> 111111111### </td>
<td> 11111### </td>
<td> 11# </td>
<td> 11##</td>
<td> 11##</td>
<td> 111111####</td>
<td> 11#</td>
</tr>
<tr>
<td> 10 </td>
<td> 11 </td>
<td> 12 </td>
<td> 13 </td>
<td> 14 </td>
<td> 15 </td>
<td> 16 </td>
<td> 17 </td>
<td> 18 </td>
</tr>
<TR BGCOLOR="#FFFFCC">
</td>
<td> 11##</td>
<td>111111111####</td>
<td>11#####</td>
<td>111111###</td>
<td>111###</td>
<td>1##</td>
<td>1111####</td>
<td>1#</td>
<td>111111####</td>
</tr>
</table>
</center>
<br>
<p>
We show tables of what the various
registers show after different parts of a run of a universal program
<i>u</i> on this program 1# + <i>write</i>.
We hasten to add that the universal program exhibited above
does <i>not</i> conform to the tables. (This is because
registers work differently in it.  Also,  the program
above is designed to work even without our simplifying
assumption.)
<br><p> 
Here are the tables; we'll have some words below about them.
<br>
 <left>
<TABLE BORDER  Cellpadding="3">
 <TR><TD BGCOLOR="#B0E0E6">R1: the input program <i>p</i>  </TD>   
<TD BGCOLOR="#FF8COO">   <i>p</i>  </TD>
         <TD BGCOLOR="#F5F5DC">  <i>p</i> </td>
         <TD BGCOLOR="#66FF99">  <i>p</i> </td>
         <TD BGCOLOR="#F5F5DC">  <i>p</i> </td>
         <TD BGCOLOR="#66FF99">  <i>p</i> </td>
         <TD BGCOLOR="#F5F5DC">  <i>p</i> </td>
         <TD BGCOLOR="#66FF99">  <i>p</i> </td>
         <TD BGCOLOR="#F5F5DC">  <i>p</i> </td>
         <TD BGCOLOR="#66FF99">  <i>p</i> </td>
         <TD BGCOLOR="#F5F5DC">  <i>p</i> </td>
         <TD BGCOLOR="#66FF99">  <i>p</i> </td>
         <TD BGCOLOR="#F5F5DC">  <i>p</i> </td>
         <td BGCOLOR="#66FF99">  <i>p</i> </td>
</tr>
 <TR><TD BGCOLOR="#B0E0E6">R2: an instruction number 1<sup>n</sup>
        </TD>   
<TD BGCOLOR="#FF8COO"> </TD>
         <TD BGCOLOR="#F5F5DC">  1 </td>
         <td BGCOLOR="#66FF99">  1 </td>
         <TD BGCOLOR="#F5F5DC">  11 </td>
         <td BGCOLOR="#66FF99">  11 </td>
         <TD BGCOLOR="#F5F5DC">  11111 </td>
         <td BGCOLOR="#66FF99">  11111 </td>
         <TD BGCOLOR="#F5F5DC">  1<sup>6</sup> </td>
         <td BGCOLOR="#66FF99">  1<sup>6</sup> </td>
         <TD BGCOLOR="#F5F5DC">  1<sup>7</sup> </td>
         <td BGCOLOR="#66FF99">  1<sup>7</sup> </td>
         <TD BGCOLOR="#F5F5DC">  1<sup>8</sup> </td>
         <td BGCOLOR="#66FF99">  1<sup>8</sup> </td>
</tr>
 <TR><TD BGCOLOR="#B0E0E6">R3: the <i>n</i>th instruction of <i>p</i>    
<TD BGCOLOR="#FF8COO"> </TD>
         <TD BGCOLOR="#F5F5DC">   </td>
         <td BGCOLOR="#66FF99"> 1##  </td>
         <TD BGCOLOR="#F5F5DC">    </td>
         <td BGCOLOR="#66FF99"> 1#####  </td>
         <TD BGCOLOR="#F5F5DC">    </td>
         <td BGCOLOR="#66FF99"> 11#  </td>
         <TD BGCOLOR="#F5F5DC">    </td>
         <td BGCOLOR="#66FF99"> 11##  </td>
         <TD BGCOLOR="#F5F5DC">    </td>
         <td BGCOLOR="#66FF99"> 11##  </td>
         <TD BGCOLOR="#F5F5DC">    </td>
         <td BGCOLOR="#66FF99"> 1<sup>6</sup> ####    </td>
   </tr>
 <TR><TD BGCOLOR="#B0E0E6">R4: the contents of R1   </TD>   
<TD BGCOLOR="#FF8COO"> </TD>
         <TD BGCOLOR="#F5F5DC">    </td>
         <td BGCOLOR="#66FF99">    </td>
         <TD BGCOLOR="#F5F5DC"> #   </td>
         <td BGCOLOR="#66FF99"> #   </td>
         <TD BGCOLOR="#F5F5DC">    </td>
         <td BGCOLOR="#66FF99">    </td>
         <TD BGCOLOR="#F5F5DC">   </td>
         <td BGCOLOR="#66FF99">    </td>
         <TD BGCOLOR="#F5F5DC">   </td>
         <td BGCOLOR="#66FF99">    </td>
         <TD BGCOLOR="#F5F5DC">    </td>
         <td BGCOLOR="#66FF99">    </td>
</tr>
 <TR><TD BGCOLOR="#B0E0E6">R5: the contents of R2  </TD>   
<TD BGCOLOR="#FF8COO"> </TD>
         <TD BGCOLOR="#F5F5DC">     </td>
         <td BGCOLOR="#66FF99">    </td>
         <TD BGCOLOR="#F5F5DC">     </td>
         <td BGCOLOR="#66FF99">    </td>
         <TD BGCOLOR="#F5F5DC">     </td>
         <td BGCOLOR="#66FF99">    </td>
         <TD BGCOLOR="#F5F5DC">  1   </td>
         <td BGCOLOR="#66FF99"> 	1   </td>
         <TD BGCOLOR="#F5F5DC">  1#   </td>
         <td BGCOLOR="#66FF99"> 	1#   </td>
         <TD BGCOLOR="#F5F5DC">  1##   </td>
         <td BGCOLOR="#66FF99">  1##  </td>
</tr>
 <TR><TD BGCOLOR="#B0E0E6">R6: the contents of R3  </TD>   
<TD BGCOLOR="#FF8COO"> </TD>
         <TD BGCOLOR="#F5F5DC">  </td>
         <td BGCOLOR="#66FF99">    </td>
         <TD BGCOLOR="#F5F5DC">  </td>
         <td BGCOLOR="#66FF99">    </td>
         <TD BGCOLOR="#F5F5DC">  </td>
         <td BGCOLOR="#66FF99">    </td>
         <TD BGCOLOR="#F5F5DC">  </td>
         <td BGCOLOR="#66FF99">    </td>
         <TD BGCOLOR="#F5F5DC">  </td>
         <td BGCOLOR="#66FF99">    </td>
         <TD BGCOLOR="#F5F5DC">  </td>
         <td BGCOLOR="#66FF99">    </td>
</tr>
</table>
</left>

<br>
continuing
<br>
 <left>
<TABLE BORDER  Cellpadding="3">
 <TR><TD BGCOLOR="#B0E0E6">R1: the input program <i>p</i>  </TD>   
         <TD BGCOLOR="#F5F5DC">  <i>p</i> </td>
         <TD BGCOLOR="#66FF99">  <i>p</i> </td>
         <TD BGCOLOR="#F5F5DC">  <i>p</i> </td>
         <TD BGCOLOR="#66FF99">  <i>p</i> </td>
         <TD BGCOLOR="#F5F5DC">  <i>p</i> </td>
         <TD BGCOLOR="#66FF99">  <i>p</i> </td>
         <TD BGCOLOR="#F5F5DC">  <i>p</i> </td>
         <TD BGCOLOR="#66FF99">  <i>p</i> </td>
         <TD BGCOLOR="#F5F5DC">  <i>p</i> </td>
         <TD BGCOLOR="#66FF99">  <i>p</i> </td>
         <TD BGCOLOR="#F5F5DC">  <i>p</i> </td>
         <td BGCOLOR="#66FF99">  <i>p</i> </td>
</tr>
 <TR><TD BGCOLOR="#B0E0E6">R2: an instr number 1<sup>n</sup>
        </TD>   
         <TD BGCOLOR="#F5F5DC">  11 </td>
         <td BGCOLOR="#66FF99">  11 </td>
         <TD BGCOLOR="#F5F5DC">  111 </td>
         <td BGCOLOR="#66FF99">  111 </td>
         <TD BGCOLOR="#F5F5DC">  1<sup>12</sup> </td>
         <td BGCOLOR="#66FF99">  1<sup>12</sup> </td>
         <TD BGCOLOR="#F5F5DC">  1<sup>14</sup> </td>
         <td BGCOLOR="#66FF99">  1<sup>14</sup> </td>
         <TD BGCOLOR="#F5F5DC">  1<sup>17</sup> </td>
         <td BGCOLOR="#66FF99">  1<sup>17</sup> </td>
         <TD BGCOLOR="#F5F5DC">  1<sup>18</sup> </td>
         <td BGCOLOR="#66FF99">  1<sup>18</sup> </td>
</tr>
 <TR><TD BGCOLOR="#B0E0E6">R3: the <i>n</i>th instruction of <i>p</i>    
         <TD BGCOLOR="#F5F5DC">   </td>
         <td BGCOLOR="#66FF99"> 1#####  </td>
         <TD BGCOLOR="#F5F5DC">    </td>
         <td BGCOLOR="#66FF99"> 1<sup>9</sup> ###  </td>
         <TD BGCOLOR="#F5F5DC">    </td>
         <td BGCOLOR="#66FF99"> 11#####  </td>
         <TD BGCOLOR="#F5F5DC">    </td>
         <td BGCOLOR="#66FF99"> 111###  </td>
         <TD BGCOLOR="#F5F5DC">    </td>
         <td BGCOLOR="#66FF99"> 1#  </td>
         <TD BGCOLOR="#F5F5DC">    </td>
         <td BGCOLOR="#66FF99"> 1<sup>6</sup> ####    </td>
   </tr>
 <TR><TD BGCOLOR="#B0E0E6">R4: the contents of R1   </TD>   
         <TD BGCOLOR="#F5F5DC">    </td>
         <td BGCOLOR="#66FF99">    </td>
         <TD BGCOLOR="#F5F5DC">    </td>
         <td BGCOLOR="#66FF99">   </td>
         <TD BGCOLOR="#F5F5DC">    </td>
         <td BGCOLOR="#66FF99">    </td>
         <TD BGCOLOR="#F5F5DC">   </td>
         <td BGCOLOR="#66FF99">    </td>
         <TD BGCOLOR="#F5F5DC">   </td>
         <td BGCOLOR="#66FF99">    </td>
         <TD BGCOLOR="#F5F5DC">  1  </td>
         <td BGCOLOR="#66FF99">  1  </td>
</tr>
 <TR><TD BGCOLOR="#B0E0E6">R5: the contents of R2  </TD>   
         <TD BGCOLOR="#F5F5DC">  1##   </td>
         <td BGCOLOR="#66FF99">  1##  </td>
         <TD BGCOLOR="#F5F%DC"> 1##    </td>
         <td BGCOLOR="#66FF99">  1##  </td>
         <TD BGCOLOR="#F5F5DC"> 1##    </td>
         <td BGCOLOR="#66FF99"> 1##   </td>
         <TD BGCOLOR="#F5F5DC">  1##   </td>
         <td BGCOLOR="#66FF99"> 	1##   </td>
         <TD BGCOLOR="#F5F5DC">  ##   </td>
         <td BGCOLOR="#66FF99"> 	##   </td>
         <TD BGCOLOR="#F5F5DC">  ##   </td>
         <td BGCOLOR="#66FF99">  ##  </td>
</tr>
 <TR><TD BGCOLOR="#B0E0E6">R6: the contents of R3  </TD>   
         <TD BGCOLOR="#F5F5DC">  </td>
         <td BGCOLOR="#66FF99">    </td>
         <TD BGCOLOR="#F5F5DC">  </td>
         <td BGCOLOR="#66FF99">    </td>
         <TD BGCOLOR="#F5F5DC">  </td>
         <td BGCOLOR="#66FF99">    </td>
         <TD BGCOLOR="#F5F5DC">  </td>
         <td BGCOLOR="#66FF99">    </td>
         <TD BGCOLOR="#F5F5DC">  </td>
         <td BGCOLOR="#66FF99">    </td>
         <TD BGCOLOR="#F5F5DC">  </td>
         <td BGCOLOR="#66FF99">    </td>
</tr>
</table>
</left>
<br>
and going further in the <i>move<sub>2,1</sub></i> part
at the end of <i>write</i>
 <left>
<TABLE BORDER  Cellpadding="3">
 <TR><TD BGCOLOR="#B0E0E6">R1: the input program <i>p</i>  </TD>   
         <TD BGCOLOR="#F5F5DC">  <i>p</i> </td>
         <TD BGCOLOR="#66FF99">  <i>p</i> </td>
         <TD BGCOLOR="#F5F5DC">  <i>p</i> </td>
         <TD BGCOLOR="#66FF99">  <i>p</i> </td>
         <TD BGCOLOR="#F5F5DC">  <i>p</i> </td>
         <TD BGCOLOR="#66FF99">  <i>p</i> </td>
         <TD BGCOLOR="#F5F5DC">  <i>p</i> </td>
         <TD BGCOLOR="#66FF99">  <i>p</i> </td>
         <TD BGCOLOR="#F5F5DC">  <i>p</i> </td>
         <TD BGCOLOR="#66FF99">  <i>p</i> </td>
         <TD BGCOLOR="#F5F5DC">  <i>p</i> </td>
         <td BGCOLOR="#66FF99">  <i>p</i> </td>
</tr>
 <TR><TD BGCOLOR="#B0E0E6">R2: an instruction number 1<sup>n</sup>
        </TD>   
         <TD BGCOLOR="#F5F5DC">  1<sup>12</sup> </td>
         <td BGCOLOR="#66FF99">  1<sup>12</sup> </td>
         <TD BGCOLOR="#F5F5DC">  1<sup>15</sup> </td>
         <td BGCOLOR="#66FF99">  1<sup>15</sup> </td>
         <TD BGCOLOR="#F5F5DC">  1<sup>16</sup> </td>
         <td BGCOLOR="#66FF99">  1<sup>16</sup> </td>
         <TD BGCOLOR="#F5F5DC">  1<sup>12</sup> </td>
         <td BGCOLOR="#66FF99">  1<sup>12</sup> </td>
         <TD BGCOLOR="#F5F5DC">  1<sup>15</sup> </td>
         <td BGCOLOR="#66FF99">  1<sup>15</sup> </td>
         <TD BGCOLOR="#F5F5DC">  1<sup>16</sup> </td>
         <td BGCOLOR="#66FF99">  1<sup>16</sup> </td>
</tr>
 <TR><TD BGCOLOR="#B0E0E6">R3: the <i>n</i>th instruction of <i>p</i>    
         <TD BGCOLOR="#F5F5DC">   </td>
         <td BGCOLOR="#66FF99"> 11#####  </td>
         <TD BGCOLOR="#F5F5DC">    </td>
         <td BGCOLOR="#66FF99"> 1##  </td>
         <TD BGCOLOR="#F5F5DC">    </td>
         <td BGCOLOR="#66FF99"> 1<sup>4</sup>####  </td>
         <TD BGCOLOR="#F5F5DC">   </td>
         <td BGCOLOR="#66FF99"> 11#####  </td>
         <TD BGCOLOR="#F5F5DC">    </td>
         <td BGCOLOR="#66FF99"> 1##  </td>
         <TD BGCOLOR="#F5F5DC">    </td>
         <td BGCOLOR="#66FF99"> 1<sup>4</sup>####  </td>
   </tr>
 <TR><TD BGCOLOR="#B0E0E6">R4: the contents of R1   </TD>   
         <TD BGCOLOR="#F5F5DC">  1  </td>
         <td BGCOLOR="#66FF99">  1  </td>
         <TD BGCOLOR="#F5F5DC">  1  </td>
         <td BGCOLOR="#66FF99"> 1  </td>
         <TD BGCOLOR="#F5F5DC">   1#  </td>
         <td BGCOLOR="#66FF99">  1#  </td>
         <TD BGCOLOR="#F5F5DC"> 1#  </td>
         <td BGCOLOR="#66FF99">  1#  </td>
         <TD BGCOLOR="#F5F5DC"> 1#  </td>
         <td BGCOLOR="#66FF99">  1#  </td>
         <TD BGCOLOR="#F5F5DC">  1##  </td>
         <td BGCOLOR="#66FF99">  1##  </td>
</tr>
 <TR><TD BGCOLOR="#B0E0E6">R5: the contents of R2  </TD>   
         <TD BGCOLOR="#F5F5DC">  ##   </td>
         <td BGCOLOR="#66FF99">  ##  </td>
         <TD BGCOLOR="#F5F5DC"> #    </td>
         <td BGCOLOR="#66FF99">  #  </td>
         <TD BGCOLOR="#F5F5DC"> #    </td>
         <td BGCOLOR="#66FF99"> #   </td>
         <TD BGCOLOR="#F5F5DC">  #  </td>
         <td BGCOLOR="#66FF99"> 	#   </td>
         <TD BGCOLOR="#F5F5DC">     </td>
         <td BGCOLOR="#66FF99"> 	  </td>
         <TD BGCOLOR="#F5F5DC">     </td>
         <td BGCOLOR="#66FF99">    </td>
</tr>
 <TR><TD BGCOLOR="#B0E0E6">R6: the contents of R3  </TD>   
         <TD BGCOLOR="#F5F5DC">  </td>
         <td BGCOLOR="#66FF99">    </td>
         <TD BGCOLOR="#F5F5DC">  </td>
         <td BGCOLOR="#66FF99">    </td>
         <TD BGCOLOR="#F5F5DC">  </td>
         <td BGCOLOR="#66FF99">    </td>
         <TD BGCOLOR="#F5F5DC">  </td>
         <td BGCOLOR="#66FF99">    </td>
         <TD BGCOLOR="#F5F5DC">  </td>
         <td BGCOLOR="#66FF99">    </td>
         <TD BGCOLOR="#F5F5DC">  </td>
         <td BGCOLOR="#66FF99">    </td>
</tr>
</table>
</left>
<br>
<p>
and concluding
<br>
 <Left>
<TABLE BORDER  Cellpadding="3">
 <TR><TD BGCOLOR="#B0E0E6">R1: the input program <i>p</i>  </TD>   
         <TD BGCOLOR="#F5F5D">  <i>p</i> </td>
         <TD BGCOLOR="#66FF99">  <i>p</i> </td>
         <TD BGCOLOR="#F5F5DC">  <i>p</i> </td>
         <TD BGCOLOR="#66FF99">  <i>p</i> </td>
         <TD BGCOLOR="#F5F5DC">  <i>p</i> </td>
         <TD BGCOLOR="#FFFFOO">  1## </td>
</tr>
 <TR><TD BGCOLOR="#B0E0E6">R2: an instruction number 1<sup>n</sup>
        </TD>   
         <TD BGCOLOR="#F5F5DC">  1<sup>12</sup> </td>
         <td BGCOLOR="#66FF99">  1<sup>12</sup> </td>
         <TD BGCOLOR="#F5F5DC">  1<sup>13</sup> </td>
         <td BGCOLOR="#66FF99">  1<sup>13</sup> </td>
         <TD BGCOLOR="#F5F5DC">  1<sup>19</sup> </td>
         <TD BGCOLOR="#FFFFOO"> </td>
</tr>
 <TR><TD BGCOLOR="#B0E0E6">R3: the <i>n</i>th instruction of <i>p</i>    
         <TD BGCOLOR="#F5F5DC">   </td>
         <td BGCOLOR="#66FF99"> 11#####  </td>
         <TD BGCOLOR="#F5F5DC">    </td>
         <td BGCOLOR="#66FF99"> 1<sup>6 ###  </td>
         <TD BGCOLOR="#F5F5DC">    </td>
         <TD BGCOLOR="#FFFFOO">    </td>
   </tr>
 <TR><TD BGCOLOR="#B0E0E6">R4: the contents of R1   </TD>   
         <TD BGCOLOR="#F5F5DC">  1##  </td>
         <td BGCOLOR="#66FF99">  1##  </td>
         <TD BGCOLOR="#F5F5DC">  1##  </td>
         <td BGCOLOR="#66FF99">  1##  </td>
         <TD BGCOLOR="#F5F5DC">  1##  </td>
                  <TD BGCOLOR="#FFFFOO">    </td>
</tr>
 <TR><TD BGCOLOR="#B0E0E6">R5: the contents of R2  </TD>   
         <TD BGCOLOR="#F5F5DC">    </td>
         <td BGCOLOR="#66FF99">    </td>
         <TD BGCOLOR="#F5F5DC">    </td>
         <td BGCOLOR="#66FF99">    </td>
         <TD BGCOLOR="#F5F5DC">    </td>
          <TD BGCOLOR="#FFFFOO">    </td>

</tr>
 <TR><TD BGCOLOR="#B0E0E6">R6: the contents of R3  </TD>   
         <TD BGCOLOR="#F5F5DC">  </td>
         <td BGCOLOR="#66FF99">    </td>
         <TD BGCOLOR="#F5F5DC">  </td>
         <td BGCOLOR="#66FF99">    </td>
         <TD BGCOLOR="#F5F5DC">  </td>
         <TD BGCOLOR="#FFFFOO">    </td>
</tr>
</table>
</left>
As promised, here are some words of explanation.
<br><p>
As you can see, there are four different colors of the columns.

 <CENTER>
<TABLE BORDER  Cellpadding="3">
         <TR><TD BGCOLOR="#FF8COO">orange: starting out by pre-processing the input (if desired) and putting 1 in R2    </td></tr>
         <TR><td BGCOLOR="#F5F5DC">beige: put into R3 the <i>n</i>th instruction of the program in R1, where 1<sup><i>n</i></sup> is in R2   </td></tr>
         <TR><TD BGCOLOR="#66FF99">green: execute the action in R3, but using R4 instead of R1, R5 instead of R2, and R6 instead of R3   </td></tr>
         <TR><td BGCOLOR="#FFFFOO">yellow: finishing up by moving and clearing the relevant registers  </td></tr>
</table>
</center>
<br><p>
The colors indicate the <strong>sub-programs</strong> of <i>u</i>.
The columns themselves indicate the register contents when one
of the sub-programs <strong>begins</strong>.
<br><p>
The first color shown is orange.
This  is the the easiest to explain.         The orange sub-program could be 11#. 
This indicates that at the
very beginning of the execution of
the input program, the instruction to look at first is instruction 1.  But if you decide to pre-process the input program (see below), then the orange sup-program would do this as well.
<br><p>
The beige columns alternate with the green columns.
The purpose of the beige program is to take the input program
(in R1) and the next instruction number (R2), and to
look up the appropriate  
instruction and put it into R3.  But we need to be sure that the contents
of R1 and R2 are not destroyed at the end.     The way to understand
the beige columns is that they tell the contents of the various registers
at the <strong>beginning</strong> of the operation of the "beige
sub-program".  You will need to write that sub-program.
It is a fairly straightforward modification of a program which
takes an input program <i>q</i> and a number <i>n</i> and gives the <i>n</i>th instruction of
<i>q</i>.   But you need to be
sure that the output goes in R3, and that the input is preserved.
<br><p>
The main action of the steps is done in the green columns;
more precisely, 
the columns shown are the <strong>beginnings</strong> of the
"green" sub-program, and then the results of that sub-program
are the beige columns which follow.
The exact action depends on what kind of instruction is in R3 in 
the given green column.  For example, if R3 has ```11##```, then
in the next beige column we add a ```#``` to R5 (not R2: it is R5
that models what is in R2 of the <i>real</i> machine when we run our universal
program).   We also must add 1 to  R2 in simulating the execution
of 11#, since after 11#, we go to the very next instruction.
This is how the green columns work when we execute an instruction
11#.   The other cases are left for you to think about.
Note also that each time the green sub-program runs, R3 is emptied 
at the end.
<br><p>
The last column color is the yellow one at the very end.
Before the yellow one starts up, the beige program
has determined (in some sense) that 
  our program $p$ has 18 instructions and we have 
  come to a point where we ask for instruction 19.  Thus, 
  the input program has halted.   We therefore want to
  take the contents of R1 as run by the input program
  (this is the word in R4), and  move this to R1.
  Also, we would like to clear out the contents of
  whichever registers are not empty.  We do this so that 
  the program we are writing will itself come to a good halt
  on its input.
<br>
<p>
It might seem silly to have R1 unchanged throughout.
But this isn't really what is happening at all.
In the course of the beige subprograms, parts of R1 are copied
to other registers, and the copied back.   So at the 
<i>end</i> of the beige sub-programs, we have the original
program <i>p</i> back in R1.   
<br><p>
Similarly, it might seem weird to have R6 completely unused
in running a universal program on <i>p</i>.  This is due
to the fact that our program <i>p</i> only uses R1 and R2.
If we worked with an example program that used R3, then R6
would not have remained empty in the tables.






## Coding unboundedly many registers into one

The simplified form of the universal program that we just saw could be modified to work on programs with any fixed finite number of registers.  But we want one universal programs that works on *all* programs, without a pre-established bound.  For this, we need a new idea.  

We now call upon a technique introduced in a separate section, the technique of two-by-two encoding.  We add an *end of register marker* to R1, and the work with a simple encoding of symbols by pairs.   We took the end of register marker to be ```##```, and we encoded ```1``` by ```11``` and ```#``` by ```1#```.  

All of that work was for R1 alone.   But we could do the same work for all of the other registers.  

Following this, the idea is to simply run all of the encoded registers into one.  



For an example of what we mean, suppose that we were dealing with the following snapshot:

<CENTER>
<TABLE BORDER  Cellpadding="3">
 <TR><TD BGCOLOR="#B0E0E6">R1: 11# </td>  
  <TD BGCOLOR="#B0E0E6">R2:  </td>  
 <TD BGCOLOR="#B0E0E6">R3: ## </td>  
 <TD BGCOLOR="#B0E0E6">R4:   </td>  
   <TD BGCOLOR="#B0E0E6">R5:   </td>  
 <TD BGCOLOR="#B0E0E6">R6: 1 </td>  
 <TD BGCOLOR="#B0E0E6">R7: #</td>  
  <TD BGCOLOR="#B0E0E6">R8: ##1 </td> 
  </tr>
</table>
</center>

(We understand also that all of the other registers are empty.)

Encoding things as we have explained allows us to code the eight registers, as in the table below:

<CENTER>
<TABLE BORDER  Cellpadding="3">
 <TR> <td> Contents </td>
 <TD BGCOLOR="#B0E0E6">R1: 11# </td>  
  <TD BGCOLOR="#B0E0E6">R2:  &nbsp;&nbsp;&nbsp; </td>  
 <TD BGCOLOR="#B0E0E6">R3: ## </td>  
 <TD BGCOLOR="#B0E0E6">R4:   </td>  
   <TD BGCOLOR="#B0E0E6">R5:   </td>  
 <TD BGCOLOR="#B0E0E6">R6: 1 </td>  
 <TD BGCOLOR="#B0E0E6">R7: #</td>  
  <TD BGCOLOR="#B0E0E6">R8: ##1 </td> 
  </tr>
   <TR><td>Encoded as</td>
   <TD BGCOLOR="#FFFFCC">11111###</td>  
<TD BGCOLOR="#FFFFCC">## </td>  
<TD BGCOLOR="#FFFFCC">1#1### </td>  
<TD BGCOLOR="#FFFFCC">##</td>  
<TD BGCOLOR="#FFFFCC">##</td>  
<TD BGCOLOR="#FFFFCC">11## </td>  
<TD BGCOLOR="#FFFFCC">1# ##</td>  
<TD BGCOLOR="#FFFFCC">1#1#11## </td> 
</table>
</center>

One important point is that the empty registers still get an end-of-register marker ```##```.

Therefore we would encode the contents of these registers by the single word

```11 11 1# ## ## 1# 1# ## ## ## 11 ## 1# ## 1# 1# 11 ##```

Without spaces, this is

```11111#####1#1#######11##1###1#1#11##```

The point is that with some additional work, we can use this single word as an encoded form of *all* the registers in our program.

<img src="https://github.com/lmoss/onesharp/blob/main/drum.jpg?raw=1" width="200" height="160">


(content:firstGroupProject)=
# Group project

```{dropdown} Click here for a suggestion on how groups could work together to write a universal program.


Every instructor and every class will be different, here is one plan that has worked for me.

Make groups of 4 or 5 people.  If someone prefers to work alone,
then it would be ok to do so, but I think it would be better to work with others.
<p>
<br>
<p>
Here is what each group needs to do:
<ol>
<li>   Make a plan on how to divide up the work of writing the
different sub-programs.   The parts are not all equally hard.
The beige program could be the hardest, especially if you follow 
my comment below on it.

<li>  You will need to meet, or at least to email each other, so that
you can be sure that the different sub-programs do what they are supposed to.

<li>  You also will need to have a plan on how to assemble everything together.
The various sub-programs will need to "talk to each other"
via goto statements.  
One good way is 
to use the 
Sanity parser.   And one way that probably does not work well would be 
to simply
put two flowcharts together.   
<li>  The group overall will need to turn in a program along with an explanation of
the various parts.  You should say clearly what the parts do, and also give some
examples of how they run.   If you have made flowcharts, it would be good
to hand those in as well.  
<li> You should test your overall program.  It would be good to run
your program on 1#,  on <i>self</i>, and on <i>1#1## + u</i>.
<li>  At various points, you will be tempted to worry about what would happen
if the input program isn't tidy.    (For example, what should your
program do if the input program is just 1111#### ?)
Strictly speaking, the universal program should itself diverge on this input.
But for our purposes, it's fine to ignore this point.
  You need not worry about this at all, only about the case
when the input program is tidy. 
<li>  Above, we worked with the simplifying assumption that
the input program only uses R1, R2, and R3.  You should get a program
<i>u</i> that works on all input programs.   
<li>
  It is important to write well.   
 Your goal should be to write something that could be understood by
 someone else.    
</ol>  
  
 ``` 
 

```{exercise}
What happens if we run the universal program $\uprog$ on the self-writing program $\self$?

Incidentally, running $\uprog$ on $\self$ takes 22,376,608 steps, whereas running $\self$ on empty registers takes 3,322.
```


```{exercise}
Show that the sets $K$ and $K_0$ are computably enumerable, where

$$
\begin{array}{lcl}
K & = & \{ p : [\![ p ]\!](p)\!\downarrow\}\\
K_0 & = & \{ p : [\![ p ]\!](\ )\!\downarrow\}\\
\end{array}
$$
```

```{exercise}
Write a program $\selfprog_1$ which writes itself to R1 but only uses R1 itself (no other registers). 
This exercise requires a trick, since there is no program like $\diagprog$ which only uses one register.

[Hint: you will need the technique of coding several registers into R1.]
```